# HuggingFace에서 제공하는 ViT 모델을 SageMaker를 통해 Inferentia 2에 배포

In [1]:
import boto3
import sagemaker
from sagemaker.pytorch import PyTorchModel
from sagemaker.serializers import JSONSerializer
from sagemaker.deserializers import JSONDeserializer
import json
import numpy as np
from PIL import Image
import requests
from io import BytesIO
import base64
import tarfile
import os
from datetime import datetime

# SageMaker 세션 및 기본 설정
sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name
bucket = sagemaker_session.default_bucket()

# 1. 모델 아티팩트 준비 (inference.py와 model.pt를 포함한 model.tar.gz 생성)
def create_inference_script():
    """SageMaker endpoint에서 사용할 inference.py 스크립트 생성"""
    inference_code = '''
import torch
import torch_neuronx
from transformers import ViTImageProcessor
import json
import base64
from PIL import Image
from io import BytesIO
import numpy as np

def model_fn(model_dir):
    """모델을 로드하는 함수"""
    # Neuron 컴파일된 모델 로드
    model = torch.jit.load(f"{model_dir}/model.pt")
    
    # Feature extractor 로드
    feature_extractor = ViTImageProcessor.from_pretrained('google/vit-base-patch16-224')
    
    return {"model": model, "feature_extractor": feature_extractor}

def input_fn(request_body, content_type):
    """입력 데이터를 전처리하는 함수"""
    if content_type == 'application/json':
        input_data = json.loads(request_body)
        
        # Base64로 인코딩된 이미지 디코드
        if 'image' in input_data:
            image_data = base64.b64decode(input_data['image'])
            image = Image.open(BytesIO(image_data))
        # URL에서 이미지 다운로드
        elif 'url' in input_data:
            import requests
            response = requests.get(input_data['url'])
            image = Image.open(BytesIO(response.content))
        else:
            raise ValueError("No image or url provided in request")
            
        return image
    else:
        raise ValueError(f"Unsupported content type: {content_type}")

def predict_fn(data, model_dict):
    """추론을 수행하는 함수"""
    model = model_dict["model"]
    feature_extractor = model_dict["feature_extractor"]
    
    # 이미지 전처리
    inputs = feature_extractor(images=data, return_tensors="pt")
    
    # 추론 실행
    with torch.no_grad():
        outputs = model(inputs['pixel_values'])
    
    # logits를 numpy array로 변환
    logits = outputs[0].numpy()
    
    # 가장 높은 확률의 클래스 찾기
    predicted_class_idx = logits.argmax(-1).item()
    
    # id2label 매핑 (ViT 기본 설정)
    # 실제 배포시에는 이 매핑을 별도 파일로 관리하는 것이 좋습니다
    from transformers import ViTForImageClassification
    config = ViTForImageClassification.from_pretrained('google/vit-base-patch16-224').config
    predicted_label = config.id2label[predicted_class_idx]
    
    # 상위 5개 예측 결과
    top_k = 5
    top_k_indices = logits[0].argsort()[-top_k:][::-1]
    top_k_probs = torch.nn.functional.softmax(torch.from_numpy(logits[0]), dim=0)
    
    predictions = []
    for idx in top_k_indices:
        predictions.append({
            "label": config.id2label[idx],
            "score": float(top_k_probs[idx])
        })
    
    return {
        "predicted_label": predicted_label,
        "predicted_class_idx": int(predicted_class_idx),
        "top_predictions": predictions
    }

def output_fn(prediction, content_type):
    """출력 데이터를 포맷팅하는 함수"""
    if content_type == 'application/json':
        return json.dumps(prediction)
    else:
        raise ValueError(f"Unsupported content type: {content_type}")
'''
    
    with open('inference.py', 'w') as f:
        f.write(inference_code)
    
    print("inference.py 스크립트가 생성되었습니다.")

def create_model_artifact():
    """model.tar.gz 아티팩트 생성"""
    # inference.py 생성
    create_inference_script()
    
    # tar.gz 파일 생성
    with tarfile.open('model.tar.gz', 'w:gz') as tar:
        # model.pt 파일이 현재 디렉토리에 있다고 가정
        if os.path.exists('model.pt'):
            tar.add('model.pt')
        else:
            print("Warning: model.pt 파일을 찾을 수 없습니다. 먼저 모델을 컴파일하고 저장하세요.")
        
        tar.add('inference.py')
    
    print("model.tar.gz 아티팩트가 생성되었습니다.")
    
    # S3에 업로드
    model_artifact_path = f's3://{bucket}/inferentia-vit-model/model.tar.gz'
    boto3.client('s3').upload_file('model.tar.gz', bucket, 'inferentia-vit-model/model.tar.gz')
    print(f"모델 아티팩트가 S3에 업로드되었습니다: {model_artifact_path}")
    
    return model_artifact_path

# 2. 모델 아티팩트 생성 및 S3 업로드
print("모델 아티팩트 준비 중...")
model_artifact_path = create_model_artifact()

# 3. PyTorchModel 생성
print("SageMaker 모델 생성 중...")
pytorch_model = PyTorchModel(
    model_data=model_artifact_path,
    role=role,
    entry_point='inference.py',  # 이미 tar.gz에 포함되어 있지만 명시
    image_uri='763104351884.dkr.ecr.us-west-2.amazonaws.com/pytorch-inference-neuronx:2.6.0-neuronx-py310-sdk2.23.0-ubuntu22.04',
    sagemaker_session=sagemaker_session
)

# 4. Endpoint 배포
print("Endpoint 배포 중...")
endpoint_name = f'vit-inferentia-endpoint-{datetime.now().strftime("%Y%m%d%H%M%S")}'

predictor = pytorch_model.deploy(
    initial_instance_count=1,
    instance_type='ml.inf2.xlarge',
    endpoint_name=endpoint_name,
    serializer=JSONSerializer(),
    deserializer=JSONDeserializer()
)

print(f"Endpoint가 성공적으로 배포되었습니다: {endpoint_name}")

/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/pydantic/_internal/_fields.py:192: UserWarning: Field name "json" in "MonitoringDatasetFormat" shadows an attribute in parent "Base"
  warnings.warn(


[07/02/25 07:54:56] INFO     Found credentials from IAM Role:                                   ]8;id=798270;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=922953;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/botocore/credentials.py#1132\1132]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


                    INFO     Found credentials from IAM Role:                                   ]8;id=956816;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=84526;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/botocore/credentials.py#1132\1132]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

                    INFO     Found credentials from IAM Role:                                   ]8;id=754542;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=995498;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/botocore/credentials.py#1132\1132]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

...

                    INFO     Repacking model artifact                                                  ]8;id=40041;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/sagemaker/model.py\model.py]8;;\:]8;id=309461;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/sagemaker/model.py#820\820]8;;\
                             (s3://sagemaker-us-west-2-418272795925/inferentia-vit-model/model.tar.gz)             
                             , script artifact (None), and dependencies ([]) into single tar.gz file               
                             located at                                                                            
                             s3://sagemaker-us-west-2-418272795925/pytorch-inference-neuronx-2025-07-0             
                             2-07-55-04-659/model.tar.gz. This may take some time depending on model               
                             size...                                                                               

[07/02/25 07:55:13] INFO     Creating model with name:                                              ]8;id=591877;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=952986;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/sagemaker/session.py#4105\4105]8;;\
                             pytorch-inference-neuronx-2025-07-02-07-55-13-537                                     

[07/02/25 07:55:14] INFO     Creating endpoint-config with name                                     ]8;id=264127;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=932815;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/sagemaker/session.py#6030\6030]8;;\
                             vit-inferentia-endpoint-20250702075504                                                

                    INFO     Creating endpoint with name vit-inferentia-endpoint-20250702075504     ]8;id=946128;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=115163;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/sagemaker/session.py#4852\4852]8;;\

---------!Endpoint가 성공적으로 배포되었습니다: vit-inferentia-endpoint-20250702075504


In [2]:
# 5. Predictor를 사용한 테스트
print("\n=== Endpoint 테스트 ===")

# 테스트 함수
def test_endpoint_with_url(predictor, image_url):
    """URL로부터 이미지를 테스트"""
    print(f"\nURL 테스트: {image_url}")
    
    # URL을 사용한 추론
    payload = {"url": image_url}
    result = predictor.predict(payload)
    
    print(f"예측된 라벨: {result['predicted_label']}")
    print(f"예측된 클래스 인덱스: {result['predicted_class_idx']}")
    print("\n상위 5개 예측:")
    for i, pred in enumerate(result['top_predictions'], 1):
        print(f"{i}. {pred['label']}: {pred['score']:.4f}")
    
    return result

test_url = "http://images.cocodataset.org/val2017/000000039769.jpg"
result1 = test_endpoint_with_url(predictor, test_url)


=== Endpoint 테스트 ===

URL 테스트: http://images.cocodataset.org/val2017/000000039769.jpg
예측된 라벨: Egyptian cat
예측된 클래스 인덱스: 285

상위 5개 예측:
1. Egyptian cat: 0.9371
2. tabby, tabby cat: 0.0386
3. tiger cat: 0.0145
4. lynx, catamount: 0.0033
5. Siamese cat, Siamese: 0.0007
